# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Polynomial-Regression-and-Step-Functions" data-toc-modified-id="Polynomial-Regression-and-Step-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Polynomial Regression and Step Functions</a></div>

In [11]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [10, 8]
plt.style.use('fivethirtyeight')
%matplotlib inline

import seaborn as sns
sns.set(color_codes=True)

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrix

from sklearn.preprocessing import PolynomialFeatures

import scipy.stats as stats

%load_ext version_information
%version_information numpy,scipy, pandas, sklearn, matplotlib, seaborn, statsmodels, plotly

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software,Version
Python,3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
IPython,7.4.0
OS,Darwin 14.5.0 x86_64 i386 64bit
numpy,1.11.0
scipy,0.19.0
pandas,0.20.0
sklearn,0.18.1
matplotlib,2.0.0
seaborn,0.7.1
statsmodels,0.8.0


In [2]:
!ls

Untitled.ipynb Wage.csv


In [4]:
!head -1 Wage.csv

"","year","age","sex","maritl","race","education","region","jobclass","health","health_ins","logwage","wage"


In [6]:
!tail -1 Wage.csv

"453557",2009,55,"1. Male","5. Separated","1. White","2. HS Grad","2. Middle Atlantic","1. Industrial","1. <=Good","1. Yes",4.50514997831991,90.4819133566401


In [7]:
infile = "Wage.csv"
df = pd.read_csv(infile,nrows=None,skiprows=None,sep=',')
print(df.shape)
df.head(2).append(df.tail(2))

(3000, 13)


,Unnamed: 0,year,age,sex,maritl,race,education,region,jobclass,health,health_ins,logwage,wage
0,231655,2006,18,1. Male,1. Never Married,1. White,1. < HS Grad,2. Middle Atlantic,1. Industrial,1. <=Good,2. No,4.318063,75.043154
1,86582,2004,24,1. Male,1. Never Married,1. White,4. College Grad,2. Middle Atlantic,2. Information,2. >=Very Good,2. No,4.255273,70.476020
2998,14375,2005,27,1. Male,1. Never Married,1. White,3. Some College,2. Middle Atlantic,1. Industrial,2. >=Very Good,1. Yes,4.477121,87.981033
2999,453557,2009,55,1. Male,5. Separated,1. White,2. HS Grad,2. Middle Atlantic,1. Industrial,1. <=Good,1. Yes,4.505150,90.481913


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 13 columns):
Unnamed: 0    3000 non-null int64
year          3000 non-null int64
age           3000 non-null int64
sex           3000 non-null object
maritl        3000 non-null object
race          3000 non-null object
education     3000 non-null object
region        3000 non-null object
jobclass      3000 non-null object
health        3000 non-null object
health_ins    3000 non-null object
logwage       3000 non-null float64
wage          3000 non-null float64
dtypes: float64(2), int64(3), object(8)
memory usage: 304.8+ KB


# Polynomial Regression and Step Functions

In [15]:
X2 = PolynomialFeatures(2).fit_transform(df.age.values.reshape(-1,1))
X2

array([[  1.00000000e+00,   1.80000000e+01,   3.24000000e+02],
       [  1.00000000e+00,   2.40000000e+01,   5.76000000e+02],
       [  1.00000000e+00,   4.50000000e+01,   2.02500000e+03],
       ..., 
       [  1.00000000e+00,   2.70000000e+01,   7.29000000e+02],
       [  1.00000000e+00,   2.70000000e+01,   7.29000000e+02],
       [  1.00000000e+00,   5.50000000e+01,   3.02500000e+03]])

In [31]:
X1 = PolynomialFeatures(1).fit_transform(df.age.values.reshape(-1,1))
X2 = PolynomialFeatures(2).fit_transform(df.age.values.reshape(-1,1))
X3 = PolynomialFeatures(3).fit_transform(df.age.values.reshape(-1,1))
X4 = PolynomialFeatures(4).fit_transform(df.age.values.reshape(-1,1))
X5 = PolynomialFeatures(5).fit_transform(df.age.values.reshape(-1,1))

y = (df.wage.values > 250).astype(int)
print('X4:\n', X4[:5])
print('y:\n', y[:5])

X4:
 [[  1.00000000e+00   1.80000000e+01   3.24000000e+02   5.83200000e+03
    1.04976000e+05]
 [  1.00000000e+00   2.40000000e+01   5.76000000e+02   1.38240000e+04
    3.31776000e+05]
 [  1.00000000e+00   4.50000000e+01   2.02500000e+03   9.11250000e+04
    4.10062500e+06]
 [  1.00000000e+00   4.30000000e+01   1.84900000e+03   7.95070000e+04
    3.41880100e+06]
 [  1.00000000e+00   5.00000000e+01   2.50000000e+03   1.25000000e+05
    6.25000000e+06]]
y:
 [0 0 0 0 0]


In [32]:
# Selecting a suitable degree for the polynomial of age.

In [34]:
fit_1 =  sm.GLS(df.wage, X1).fit()
fit_2 =  sm.GLS(df.wage, X2).fit()
fit_3 =  sm.GLS(df.wage, X3).fit()
fit_4 =  sm.GLS(df.wage, X4).fit()
fit_5 =  sm.GLS(df.wage, X5).fit()

sm.stats.anova_lm(fit_1, fit_2, fit_3, fit_4, fit_5, typ=1)

/Users/poudel/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in greater

/Users/poudel/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning:

invalid value encountered in less

/Users/poudel/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning:

invalid value encountered in less_equal



,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,2998.0,5.022216e+06,0.0,NaN,NaN,NaN
1,2997.0,4.793430e+06,1.0,228786.010128,143.593107,2.363850e-32
2,2996.0,4.777674e+06,1.0,15755.693664,9.888756,1.679202e-03
3,2995.0,4.771604e+06,1.0,6070.152124,3.809813,5.104620e-02
4,2994.0,4.770322e+06,1.0,1282.563017,0.804976,3.696820e-01
